# Import Dependencies

In [1]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

# Visualization
import ipyleaflet
import matplotlib.pyplot as plt
from IPython.display import Image
import seaborn as sns

# Data Science
import numpy as np
import pandas as pd
import xarray as xr

# Planetary Computer Tools
import pystac
import pystac_client
import odc
from pystac_client import Client
from pystac.extensions.eo import EOExtension as eo
from odc.stac import stac_load
import planetary_computer as pc
pc.settings.set_subscription_key('cf5657d28bb2408ba8fd775642c2e1cb')

# Others
import requests
import rich.table
from itertools import cycle
from tqdm import tqdm
tqdm.pandas()
from datetime import datetime
import os
from glob import glob
import time

# Access and Download Landsat Data

In [6]:
crop_presence_data = pd.read_csv("../data/Crop_Location_Data.csv")
crop_presence_data.head()

,Latitude and Longitude,Class of Land
0,"(10.323727047081501, 105.2516346045924)",Rice
1,"(10.322364360592521, 105.27843410554115)",Rice
2,"(10.321455902933202, 105.25254306225168)",Rice
3,"(10.324181275911162, 105.25118037576274)",Rice
4,"(10.324635504740822, 105.27389181724476)",Rice


In [53]:
# prepare bounding box
box_size_deg_3x3 = 0.0002 # Surrounding box in degrees, yields approximately 3x3 pixel region
box_size_deg_5x5 = 0.0004 # Surrounding box in degrees, yields approximately 5x5 pixel region
box_size_deg_7x7 = 0.0006 # Surrounding box in degrees, yields approximately 7x7 pixel region

def create_bbox(lat_long, box_size_deg):
    lat, lon = float(lat_long[0]), float(lat_long[1])
    min_lon = lon-box_size_deg/2
    min_lat = lat-box_size_deg/2
    max_lon = lon+box_size_deg/2
    max_lat = lat+box_size_deg/2
    return (min_lon, min_lat, max_lon, max_lat)

In [71]:
def get_landsat_data(latlong,time_slice,assets,box_size_deg=None):
    '''
    Returns VV and VH values for a given latitude and longitude 
    Attributes:
    latlong - A tuple with 2 elements - latitude and longitude
    time_slice - Timeframe for which the VV and VH values have to be extracted
    assets - A list of bands to be extracted
    '''

    latlong=latlong.replace('(','').replace(')','').replace(' ','').split(',')
    bbox_of_interest = (float(latlong[1]) , float(latlong[0]), float(latlong[1]) , float(latlong[0]))
    if box_size_deg is not None:
        bbox_of_interest= create_bbox(latlong, box_size_deg)
    time_of_interest = time_slice

    catalog = pystac_client.Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1"
    )
    search = catalog.search(
        collections=["landsat-c2-l2"],
        bbox=bbox_of_interest,
        datetime=time_of_interest,
        query={"eo:cloud_cover": {"lt": 10}},
    )
    items = list(search.get_all_items())
    bands_of_interest = assests
    data = stac_load(
        items, 
        bands=bands_of_interest, 
        patch_url=pc.sign, 
        bbox=bbox_of_interest,
    )
    return data

In [ ]:
TM b1, b2, b3, b4, b5, b7
ETM b1, b2, b3, b4, b5, b7
OLI b2, b3, b4, b5, b6, b7

SyntaxError: invalid syntax (1266592300.py, line 1)

In [72]:
# Prepare the Folder
directory="../data/train/landsat/1-year-202111-202211/b1-b7-1x1"
if not os.path.exists(directory):
    print("Creating %s" % (directory))
    os.makedirs(directory)
    print("Done")
else:
    print("%s already existed" % (directory))

# Download the Data
time_slice = "2021-11-01/2022-11-01"
assests = [
    'red', 'blue', 'green', 'nir08', 'swir16', 'swir22', 'qa_pixel'
]

for coordinates in tqdm(crop_presence_data['Latitude and Longitude']):
    data = get_landsat_data(coordinates,time_slice,assests)
    data.to_netcdf(os.path.join(directory, coordinates+".nc"))
data

../data/train/landsat/1-year-202111-202211/b1-b7-1x1 already existed


 25%|██▌       | 150/600 [00:50<02:30,  2.99it/s]


ValueError: Failed to auto-guess CRS/resolution.

In [ ]:
# Prepare the Folder
directory="../data/train/landsat/1-year-202111-202211/b1-b7-3x3"
if not os.path.exists(directory):
    print("Creating %s" % (directory))
    os.makedirs(directory)
    print("Done")
else:
    print("%s already existed" % (directory))

# Download the Data
time_slice = "2021-11-01/2022-11-01"
assests = [
    'red', 'blue', 'green', 'nir08', 'swir16', 'swir22', 'qa_pixel'
]

for coordinates in tqdm(crop_presence_data['Latitude and Longitude']):
    data = get_landsat_data(coordinates,time_slice,assests, box_size_deg_3x3)
    data.to_netcdf(os.path.join(directory, coordinates+".nc"))
data

In [ ]:
# Prepare the Folder
directory="../data/train/landsat/1-year-202111-202211/b1-b7-3x3"
if not os.path.exists(directory):
    print("Creating %s" % (directory))
    os.makedirs(directory)
    print("Done")
else:
    print("%s already existed" % (directory))

# Download the Data
time_slice = "2021-11-01/2022-11-01"
assests = [
    'red', 'blue', 'green', 'nir08', 'swir16', 'swir22', 'qa_pixel'
]

for coordinates in tqdm(crop_presence_data['Latitude and Longitude']):
    data = get_landsat_data(coordinates,time_slice,assests, box_size_deg_3x3)
    data.to_netcdf(os.path.join(directory, coordinates+".nc"))
data